In [1]:
print("你好")

你好


In [2]:
from data.data_proc_hk import get_data
from data.data_proc_hk import get_stop_words
import numpy as np
import numba as nb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.externals import joblib
import jieba

In [3]:
# 参数
data_num = 10000 # 用于训练的数据量
stop_words = get_stop_words()

@nb.jit(forceobj=True, cache=True)
def data_proc():
  all_data = get_data()
  tmp = []
  for key in all_data:
    val = all_data[key]
    tmp .append(val)
  all_data = tmp

  texts = [item[2]+item[3] for item in all_data]
  labels = [item[1] for item in all_data]
  return np.array(texts), np.array(labels)

texts, labels = data_proc()
texts = texts[:data_num]
labels = labels[:data_num]

In [4]:
def getTfIdf(all_texts):
  # 一行表示一个文本， 分词之间按照空格隔开。
  result = []
  for line in all_texts:
    tmp = list(jieba.cut(line.strip()))
    tmp = [it for it in tmp if it not in stop_words]
    tmp = " ".join(tmp)
    result.append(tmp)
  
  # 统计词频矩阵, 得到的内容 ： (句子id,  单词在词表中的id)  单词出现的次数
  freWord = CountVectorizer()
  tf = freWord.fit_transform(result)
  # 统计每个词的 tf-idf 值， 得到的内容： (句子id, 单词在词表中的id) 对应这个单词的 tf-idf 值
  tfidftrans = TfidfTransformer()
  tfidf = tfidftrans.fit_transform(tf)

  return tfidf

allX = getTfIdf(texts)

Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\hk\AppData\Local\Temp\jieba.cache


Loading model cost 1.273 seconds.


Prefix dict has been built succesfully.


In [5]:
x_train, x_test, y_train, y_test = train_test_split(allX, labels, test_size=0.25)

In [6]:
gbr = GradientBoostingClassifier(n_estimators=3000, max_depth=2, min_samples_split=2, learning_rate=0.1)
gbr.fit(x_train, y_train.ravel())
joblib.dump(gbr, 'train_model_result4.m')   # 保存模型

y_gbr = gbr.predict(x_train)
y_gbr1 = gbr.predict(x_test)
acc_train = gbr.score(x_train, y_train)
acc_test = gbr.score(x_test, y_test)
print(acc_train)
print(acc_test)

KeyboardInterrupt: 

In [ ]:
help(gbr.fit)